In [ ]:
# setting the Thesis project folder as working directory
%cd "../.."

Import packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import io
from skimage.measure import label
from skimage.measure import regionprops
import matplotlib as mpl
import wandb
import os
import sys
from utility_functions.utility import *

# Import Image

Parameters setting

In [ ]:
# plots sizes
plt.rcParams['figure.figsize'] = [12, 12]

config = {
    'dataset': 'white-mad-hatters/Team-Artifacts-Tesi/img3_test_set:v1',
}
# os.environ['WANDB_NOTEBOOK_NAME'] = 'Postprocess_2'

In [ ]:
wandb.login()

In [ ]:
X, Y = download_data(project_name='Team-Artifacts-Tesi', entity='white-mad-hatters', artifact_path=config['dataset'],) # ann_filename='annot2.npy', img_filename='green_blue_fusion2.npy'

In [ ]:
plt.imshow(X[:, :, 0])

# Find optimal treshold 

In [ ]:
plt.imshow(Y)

CELLS REGIONS

In [ ]:
cells_intens = np.where(Y == 1, X[:, :, 0], 0)
plt.imshow(cells_intens)

A detail of the above image

In [ ]:
plt.imshow(cells_intens[2000:3000, 2500:3500])

Pixels intensity distribution of cells regions

In [ ]:
hist = np.histogram(cells_intens.flatten(), bins=256)
plt.hist(x=cells_intens.flatten(), bins=hist[1][1:]) # start from one to exclude first bin (0, x)
plt.xticks(np.arange(0, 1.1, step=0.1) )
plt.show()

BACKGROUND

In [ ]:
cells_intens = np.where(Y == 0, X[:, :, 0], 0)
plt.imshow(cells_intens)

A detail of above image

In [ ]:
plt.imshow(cells_intens[2000:3000, 2500:3500])

Pixels intensity distribution: background

In [ ]:
hist = np.histogram(cells_intens.flatten(), bins=256)
plt.hist(x=cells_intens.flatten(), bins=hist[1][1:])
plt.xticks(np.arange(0, 1.1, step=0.1))
plt.show()

# Postprocess

Importing prediction image

In [ ]:
y_pred = io.imread('output/y_pred.tif') #originalimage btw 0 and 255
y_pred.shape

Intensity image

In [ ]:
intensity_img = X[:, :, 0:2]

In [ ]:
intensity_img.shape

In [ ]:
postpr_img = postprocess(y_pred, intensity_img, m_area_trsh=200, intensity_trsh=0.15, round_n_trsh=0.2)

In [ ]:
plt.imshow(y_pred[1000:2000, 3000:4000])

In [ ]:
plt.imshow(postpr_img[1000:2000, 3000:4000])

In [ ]:
plt.imshow(np.around(intensity_img[1000:2000, 3000:4000, 1]))

## Evaluation

In [ ]:
# Y = np.round(io.imread('input/Ann3.tif'))[ 1218:, :-651]
Y.shape

In [ ]:
# round predicted image to 0 and 1
y_pre_rd = np.around(postpr_img)

# create final coloured image with same shape of the predicted one
y_final = np.full((postpr_img.shape), -1, dtype=np.int8)

# assign Y
Y_new = Y#[:, :, z_chan]

# substitute tp, tn, fp, fn with a number
y_final[np.logical_and(Y_new, y_pre_rd)] = 0 # tp
y_final[np.logical_and(np.logical_not(Y_new), np.logical_not(y_pre_rd))] = 1 # tn
y_final[np.logical_and(np.logical_not(Y_new), y_pre_rd)] = 2 #fp
y_final[np.logical_and(Y_new, np.logical_not(y_pre_rd))] = 3 #fn

print(np.unique(y_final) )
tp = np.sum(y_final == 0)
tn = np.sum(y_final == 1)
fp = np.sum(y_final == 2)
fn = np.sum(y_final == 3)

# Plot the image
colors = ['green', 'blue', 'red', 'yellow']
bounds = [0,1,2,3,4]

cmap = mpl.colors.ListedColormap(colors)
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)

fig = plt.figure()
ax = plt.Axes(fig, [0., 0., 1., 1.])
ax.set_axis_off()
fig.add_axes(ax)
#plt.set_cmap(cmap)
ax.imshow(y_final, interpolation='none', cmap=cmap, norm=norm)#aspect='equal')

#plt.imshow(y_plot, interpolation='none', cmap=cmap, norm=norm)
plt.savefig('output/pp_UL_2.1-3.2.png', dpi = 620)
print(' recall: ', f'{tp/(tp+fn):.4f}', '\n',
    'precision: ',f'{tp/(tp+fp):.4f}', '\n',)
plt.show()

In [ ]:
# round predicted image to 0 and 1
y_pre_rd = np.around(postpr_img)
Y_new = Y#[:, :, Y_slice]

# create final coloured image with same shape of the predicted one
y_final = np.full((4, y_pre_rd.shape[0], y_pre_rd.shape[1]), -1, dtype=np.int8)

# substitute tp, tn, fp, fn with a number
y_final[0] = np.logical_and(Y_new, y_pre_rd).astype(int) # tp
y_final[1] = np.logical_and(np.logical_not(Y_new), np.logical_not(y_pre_rd)).astype(int) # tn
y_final[2] = np.logical_and(np.logical_not(Y_new), y_pre_rd).astype(int) #fp
y_final[3] = np.logical_and(Y_new, np.logical_not(y_pre_rd)).astype(int) #fn

print(np.unique(y_final))

In [ ]:
io.imsave('output/pp_0.25_0.3_UlL2.1-3.1.tif', y_final, photometric='minisblack')